# Bike Availability Proxy Model

This notebook builds a proxy estimate for bike availability at Columbia University's 7 Citi Bike stations using trip inflow/outflow data.

## Methodology

- **Time granularity**: 15-minute intervals
- **Initial availability**: Start at 0 (tracking net change/relative availability)
- **Inflows**: Bikes arriving (trip ends at station)
- **Outflows**: Bikes leaving (trip starts at station)
- **Missing end stations**: Counted as outflows only
- **Cumulative availability**: Running sum of (inflows - outflows)

## Validation Strategy

To assess accuracy:
1. Check for negative availability (impossible in reality)
2. Examine min/max ranges per station
3. Compare peak calculated bikes to typical station capacity (15-40 bikes)

## 1. Data Loading & Preparation

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Load filtered Columbia dataset
df = pd.read_csv('../data/columbia_filtered_citibike.csv', 
				 parse_dates=['started_at', 'ended_at'],
				 low_memory=False)

print(f"Total trips loaded: {len(df):,}")
print(f"Date range: {df['started_at'].min()} to {df['ended_at'].max()}")
print(f"\nMissing end_station_id: {df['end_station_id'].isna().sum():,} ({df['end_station_id'].isna().sum()/len(df)*100:.2f}%)")

In [ ]:
# Define Columbia stations
columbia_stations = {
	'7783.18': 'Broadway & W 122 St',
	'7741.04': 'Morningside Dr & Amsterdam Ave',
	'7745.07': 'W 120 St & Claremont Ave',
	'7727.07': 'Amsterdam Ave & W 119 St',
	'7713.11': 'W 116 St & Broadway',
	'7692.11': 'W 116 St & Amsterdam Ave',
	'7713.01': 'W 113 St & Broadway'
}

print("Columbia University Stations:")
for station_id, name in columbia_stations.items():
	print(f"  {station_id}: {name}")

## 2. Build 15-Minute Interval Availability Proxy

In [ ]:
# Create time bins at 15-minute intervals
time_start = df['started_at'].min().floor('15min')
time_end = df['ended_at'].max().ceil('15min')

print(f"Creating time bins from {time_start} to {time_end}")
print(f"Interval: 15 minutes")

# Generate full time range
time_bins = pd.date_range(start=time_start, end=time_end, freq='15min')
print(f"Total time bins: {len(time_bins):,}")

In [ ]:
# Prepare outflow data (bikes leaving stations)
# All trips have start_station_id
outflows = df[df['start_station_id'].isin(columbia_stations.keys())].copy()
outflows['time_bin'] = outflows['started_at'].dt.floor('15min')

outflow_counts = outflows.groupby(['time_bin', 'start_station_id']).size().reset_index(name='outflow')
outflow_counts.rename(columns={'start_station_id': 'station_id'}, inplace=True)

print(f"Outflow records: {len(outflow_counts):,}")
print(f"Total bikes leaving Columbia stations: {outflow_counts['outflow'].sum():,}")

In [ ]:
# Prepare inflow data (bikes arriving at stations)
# Only count trips with valid end_station_id (missing ones counted as outflows only)
inflows = df[(df['end_station_id'].isin(columbia_stations.keys())) & 
			 (df['end_station_id'].notna())].copy()
inflows['time_bin'] = inflows['ended_at'].dt.floor('15min')

inflow_counts = inflows.groupby(['time_bin', 'end_station_id']).size().reset_index(name='inflow')
inflow_counts.rename(columns={'end_station_id': 'station_id'}, inplace=True)

print(f"Inflow records: {len(inflow_counts):,}")
print(f"Total bikes arriving at Columbia stations: {inflow_counts['inflow'].sum():,}")

In [ ]:
# Create complete time x station grid
stations_df = pd.DataFrame(list(columbia_stations.keys()), columns=['station_id'])
time_df = pd.DataFrame(time_bins, columns=['time_bin'])

# Cross join to get all combinations
availability_df = time_df.merge(stations_df, how='cross')

print(f"Complete grid size: {len(availability_df):,} rows ({len(time_bins):,} time bins × {len(columbia_stations)} stations)")

In [ ]:
# Merge inflows and outflows
availability_df = availability_df.merge(inflow_counts, on=['time_bin', 'station_id'], how='left')
availability_df = availability_df.merge(outflow_counts, on=['time_bin', 'station_id'], how='left')

# Fill NaN with 0 (no activity in that interval)
availability_df['inflow'] = availability_df['inflow'].fillna(0).astype(int)
availability_df['outflow'] = availability_df['outflow'].fillna(0).astype(int)

# Calculate net change
availability_df['net_change'] = availability_df['inflow'] - availability_df['outflow']

print("Sample of merged data:")
display(availability_df.head(20))

In [ ]:
# Calculate cumulative availability per station (starting from 0)
availability_df = availability_df.sort_values(['station_id', 'time_bin'])
availability_df['cumulative_availability'] = availability_df.groupby('station_id')['net_change'].cumsum()

# Add station names
availability_df['station_name'] = availability_df['station_id'].map(columbia_stations)

print("\nAvailability calculation complete!")
print(f"Total records: {len(availability_df):,}")
display(availability_df.head(30))

## 3. Validation & Accuracy Assessment

In [ ]:
# Check for negative availability (impossible in reality)
negative_availability = availability_df[availability_df['cumulative_availability'] < 0]

print("=" * 80)
print("VALIDATION CHECK 1: Negative Availability")
print("=" * 80)
print(f"\nIntervals with negative availability: {len(negative_availability):,} / {len(availability_df):,} ({len(negative_availability)/len(availability_df)*100:.2f}%)\n")

if len(negative_availability) > 0:
	print("\nNegative availability by station:")
	neg_by_station = negative_availability.groupby('station_name').agg({
		'cumulative_availability': ['count', 'min', 'mean'],
		'time_bin': ['min', 'max']
	}).round(2)
	display(neg_by_station)
	
	print("\n⚠️  WARNING: Negative availability indicates the proxy model has limitations.")
	print("    This could be due to:")
	print("    - Starting at 0 bikes (stations likely had bikes on Jan 1, 2024)")
	print("    - Manual rebalancing operations not captured in trip data")
	print("    - Bikes removed for maintenance")
else:
	print("✓ No negative availability detected - proxy model is consistent!")

In [ ]:
# Min/Max analysis per station
print("=" * 80)
print("VALIDATION CHECK 2: Min/Max Availability per Station")
print("=" * 80)

station_stats = availability_df.groupby('station_name').agg({
	'cumulative_availability': ['min', 'max', 'mean', 'std'],
	'inflow': 'sum',
	'outflow': 'sum'
}).round(2)

station_stats.columns = ['Min Bikes', 'Max Bikes', 'Mean Bikes', 'Std Dev', 'Total Inflows', 'Total Outflows']
station_stats['Range'] = station_stats['Max Bikes'] - station_stats['Min Bikes']
station_stats['Net Flow'] = station_stats['Total Inflows'] - station_stats['Total Outflows']

station_stats = station_stats.sort_values('Max Bikes', ascending=False)
display(station_stats)

print("\n📊 Typical Citi Bike station capacity: 15-40 bikes")
print(f"\n   Stations with Max > 40 bikes: {len(station_stats[station_stats['Max Bikes'] > 40])}")
print(f"   Stations with Max > 60 bikes: {len(station_stats[station_stats['Max Bikes'] > 60])}")

In [ ]:
# Peak capacity analysis
print("=" * 80)
print("VALIDATION CHECK 3: Peak Capacity Analysis")
print("=" * 80)

for station_id, station_name in columbia_stations.items():
	station_data = availability_df[availability_df['station_id'] == station_id]
	
	min_bikes = station_data['cumulative_availability'].min()
	max_bikes = station_data['cumulative_availability'].max()
	mean_bikes = station_data['cumulative_availability'].mean()
	
	min_time = station_data[station_data['cumulative_availability'] == min_bikes]['time_bin'].iloc[0]
	max_time = station_data[station_data['cumulative_availability'] == max_bikes]['time_bin'].iloc[0]
	
	print(f"\n{station_name}:")
	print(f"  Min: {min_bikes:6.0f} bikes at {min_time}")
	print(f"  Max: {max_bikes:6.0f} bikes at {max_time}")
	print(f"  Mean: {mean_bikes:5.1f} bikes")
	print(f"  Range: {max_bikes - min_bikes:.0f} bikes")
	
	if max_bikes > 40:
		print(f"  ⚠️  Max exceeds typical capacity (40 bikes)")
	if min_bikes < -20:
		print(f"  ⚠️  Large negative value suggests missing initial capacity")

In [ ]:
# Summary statistics
print("=" * 80)
print("OVERALL SUMMARY")
print("=" * 80)

print(f"\nDataset Coverage:")
print(f"  Time range: {availability_df['time_bin'].min()} to {availability_df['time_bin'].max()}")
print(f"  Total days: {(availability_df['time_bin'].max() - availability_df['time_bin'].min()).days}")
print(f"  Total intervals: {len(time_bins):,} (15-min each)")

print(f"\nTrip Activity:")
print(f"  Total inflows: {availability_df['inflow'].sum():,}")
print(f"  Total outflows: {availability_df['outflow'].sum():,}")
print(f"  Net system change: {availability_df['inflow'].sum() - availability_df['outflow'].sum():,}")

print(f"\nAvailability Ranges:")
print(f"  Overall min: {availability_df['cumulative_availability'].min():.0f} bikes")
print(f"  Overall max: {availability_df['cumulative_availability'].max():.0f} bikes")
print(f"  System-wide mean: {availability_df['cumulative_availability'].mean():.1f} bikes")

## 4. Visualizations

In [ ]:
# Time series plot - All stations
fig = px.line(availability_df, 
			  x='time_bin', 
			  y='cumulative_availability',
			  color='station_name',
			  title='Bike Availability Proxy Over Time (15-min intervals)<br>All Columbia Stations',
			  labels={
				  'time_bin': 'Date',
				  'cumulative_availability': 'Estimated Bikes Available',
				  'station_name': 'Station'
			  },
			  height=600)

fig.add_hline(y=0, line_dash="dash", line_color="red", 
			  annotation_text="0 bikes (baseline)",
			  annotation_position="right")

fig.update_layout(template='plotly_dark',
				  hovermode='x unified')

fig.show()

In [ ]:
# Individual station plots
fig = make_subplots(rows=4, cols=2,
				    subplot_titles=list(columbia_stations.values()),
				    vertical_spacing=0.08,
				    horizontal_spacing=0.1)

station_list = list(columbia_stations.items())

for idx, (station_id, station_name) in enumerate(station_list):
	row = idx // 2 + 1
	col = idx % 2 + 1
	
	station_data = availability_df[availability_df['station_id'] == station_id]
	
	fig.add_trace(
		go.Scatter(x=station_data['time_bin'],
				   y=station_data['cumulative_availability'],
				   mode='lines',
				   name=station_name,
				   showlegend=False),
		row=row, col=col
	)
	
	fig.add_hline(y=0, line_dash="dash", line_color="red", 
				  row=row, col=col, opacity=0.5)

fig.update_layout(height=1200,
				  title_text='Bike Availability by Station (15-min intervals)',
				  template='plotly_dark')

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Bikes Available")

fig.show()

In [ ]:
# Distribution of availability per station
fig = px.box(availability_df,
			 x='station_name',
			 y='cumulative_availability',
			 title='Distribution of Bike Availability by Station',
			 labels={
				 'station_name': 'Station',
				 'cumulative_availability': 'Estimated Bikes Available'
			 },
			 height=600)

fig.add_hline(y=0, line_dash="dash", line_color="red")
fig.add_hline(y=40, line_dash="dot", line_color="yellow",
			  annotation_text="Typical max capacity (40)",
			  annotation_position="right")

fig.update_layout(template='plotly_dark')
fig.update_xaxes(tickangle=-45)

fig.show()

In [ ]:
# Add time-based features for heatmap
availability_df['hour'] = availability_df['time_bin'].dt.hour
availability_df['day_of_week'] = availability_df['time_bin'].dt.dayofweek  # 0=Monday
availability_df['day_name'] = availability_df['time_bin'].dt.day_name()

# Average availability by hour and day of week
heatmap_data = availability_df.groupby(['station_name', 'day_of_week', 'hour'])['cumulative_availability'].mean().reset_index()

# Create heatmap for one station (most active)
most_active_station = station_stats.index[0]
station_heatmap = heatmap_data[heatmap_data['station_name'] == most_active_station]

pivot_data = station_heatmap.pivot(index='hour', columns='day_of_week', values='cumulative_availability')
pivot_data.columns = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig = px.imshow(pivot_data.T,
				labels=dict(x="Hour of Day", y="Day of Week", color="Avg Bikes"),
				title=f'Average Bike Availability Heatmap<br>{most_active_station}',
				aspect='auto',
				height=400,
				color_continuous_scale='RdYlGn')

fig.update_layout(template='plotly_dark')
fig.show()

In [ ]:
# Min/Max bars per station
fig = go.Figure()

stations_sorted = station_stats.sort_values('Range', ascending=True).index

for station_name in stations_sorted:
	station_data = availability_df[availability_df['station_name'] == station_name]
	min_val = station_data['cumulative_availability'].min()
	max_val = station_data['cumulative_availability'].max()
	
	fig.add_trace(go.Bar(
		x=[min_val, max_val],
		y=[station_name, station_name],
		orientation='h',
		name=station_name,
		showlegend=False
	))

fig.add_vline(x=0, line_dash="dash", line_color="red")
fig.add_vline(x=40, line_dash="dot", line_color="yellow",
			  annotation_text="Typical capacity",
			  annotation_position="top")

fig.update_layout(
	title='Min/Max Bike Availability Range by Station',
	xaxis_title='Estimated Bikes Available',
	yaxis_title='Station',
	height=500,
	template='plotly_dark',
	barmode='overlay'
)

fig.show()

## 5. Accuracy Assessment & Conclusions

In [ ]:
print("=" * 80)
print("ACCURACY ASSESSMENT: Is the Proxy Model Suitable for Predictions?")
print("=" * 80)

# Calculate assessment metrics
pct_negative = len(negative_availability) / len(availability_df) * 100
stations_exceeding_capacity = len(station_stats[station_stats['Max Bikes'] > 40])
max_negative = availability_df['cumulative_availability'].min()
max_positive = availability_df['cumulative_availability'].max()

print(f"\n1. NEGATIVE AVAILABILITY CHECK")
print(f"   - Intervals with negative values: {pct_negative:.2f}%")
if pct_negative > 10:
	print(f"   - ❌ HIGH: Over 10% of intervals show impossible negative bikes")
	print(f"   - Issue: Starting at 0 bikes doesn't match reality")
elif pct_negative > 1:
	print(f"   - ⚠️  MODERATE: Some negative values present")
	print(f"   - Likely due to missing initial capacity or rebalancing")
else:
	print(f"   - ✓ GOOD: Very few negative values")

print(f"\n2. CAPACITY REALISM CHECK")
print(f"   - Stations exceeding typical capacity (40 bikes): {stations_exceeding_capacity}/7")
print(f"   - Maximum calculated availability: {max_positive:.0f} bikes")
if max_positive > 60:
	print(f"   - ⚠️  Values significantly exceed typical capacity")
	print(f"   - Suggests cumulative drift or missing rebalancing data")
elif max_positive > 40:
	print(f"   - ~ ACCEPTABLE: Slightly above typical capacity")
else:
	print(f"   - ✓ GOOD: Values within realistic range")

print(f"\n3. RANGE & STABILITY")
print(f"   - Maximum negative: {max_negative:.0f} bikes")
print(f"   - Maximum positive: {max_positive:.0f} bikes")
print(f"   - Total range: {max_positive - max_negative:.0f} bikes")
if abs(max_negative) > 30 or max_positive > 60:
	print(f"   - ⚠️  Large range suggests cumulative error accumulation")
else:
	print(f"   - ✓ Range appears reasonable")

print(f"\n4. NET SYSTEM BALANCE")
total_inflows = availability_df['inflow'].sum()
total_outflows = availability_df['outflow'].sum()
net_system = total_inflows - total_outflows
print(f"   - Total inflows: {total_inflows:,}")
print(f"   - Total outflows: {total_outflows:,}")
print(f"   - Net system change: {net_system:,}")
print(f"   - Imbalance rate: {abs(net_system)/total_outflows*100:.2f}%")

print(f"\n" + "=" * 80)
print("FINAL ASSESSMENT")
print("=" * 80)

if pct_negative < 5 and max_positive < 50:
	print("\n✅ PROXY MODEL IS SUITABLE for prediction modeling")
	print("   The inflow/outflow proxy appears to track availability trends well.")
	print("   Recommendation: Use this as a feature for ML models.")
elif pct_negative < 15 and max_positive < 70:
	print("\n⚠️  PROXY MODEL HAS LIMITATIONS but may still be useful")
	print("   The model shows some drift from reality due to:")
	print("   - Unknown initial bike counts")
	print("   - Manual rebalancing operations not captured")
	print("   Recommendation: Consider using relative changes or adding calibration.")
else:
	print("\n❌ PROXY MODEL NEEDS SIGNIFICANT IMPROVEMENT")
	print("   High levels of negative availability or unrealistic maximums.")
	print("   Recommendation: Obtain actual availability data or add rebalancing detection.")

print("\n" + "=" * 80)

## 6. Export Data for Prediction Modeling

In [ ]:
# Export availability data for use in prediction models
output_file = '../data/bike_availability_15min.csv'
availability_df.to_csv(output_file, index=False)
print(f"Exported availability data to: {output_file}")
print(f"Rows: {len(availability_df):,}")
print(f"Columns: {list(availability_df.columns)}")